In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
from bs4 import BeautifulSoup
import seaborn as sns


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data=pd.read_csv("/amazon_reviews.csv.zip",index_col="Unnamed: 0")
data.head(10)

FileNotFoundError: ignored

In [ ]:
data=pd.DataFrame(data)
col=["review_body","star_rating"]
data=data.loc[:,col]

In [ ]:
data.head()

In [ ]:

data=data[data["star_rating"]!=3]
data["star_rating"].value_counts()

In [ ]:
data["label"]=list(map(lambda x: 1 if(x>3) else 0,data["star_rating"]))
data.head(10)

In [ ]:
data["label"].value_counts().plot(kind="bar")

In [ ]:
data["pre_process"]=data["review_body"].apply(lambda x :str(x).lower())
# Lets remove the HTML tags and URL from Review_body
data["pre_process"]=data["pre_process"].apply(lambda x: BeautifulSoup(x,"html.parser").get_text())

In [ ]:
def contraction(s):
    s=re.sub(r"won't","will not",s)
    s=re.sub(r"would't","would not",s)
    s = re.sub(r"could't", "could not",s)
    s = re.sub(r"\'d", " would",s)
    s = re.sub(r"can\'t", "can not",s)
    s = re.sub(r"n\'t", " not", s)
    s= re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    s = re.sub(r"doesn't", "does not",s)
    return s
data["pre_process"]=data["pre_process"].apply(lambda x:(contraction(x)))

In [ ]:
import nltk
nltk.download('punkt')


In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:


from nltk.stem import WordNetLemmatizer
lemma=WordNetLemmatizer()
data["pre_process"]=data["pre_process"].apply(lambda x:" ".join([lemma.lemmatize(x)for x in nltk.word_tokenize(x)]))

In [ ]:
data.head(10)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score

# bag of words and TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
X_train ,X_test,Y_train,Y_test=train_test_split(data["pre_process"],data["label"],test_size=0.3,random_state=10)
print("Train Size: ",X_train.shape,Y_train.shape,"   Test Size: ",(X_test.shape,Y_test.shape))

In [ ]:
X_train

In [ ]:
Y_train

In [ ]:
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)
tf_x_train

In [ ]:
tf_x_test

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=1000,solver="saga")
model.fit(tf_x_train,Y_train)

In [ ]:
y_pred=model.predict(tf_x_test)
from sklearn import metrics
print("Accuracy for test data :- ",round(metrics.accuracy_score(Y_test,y_pred),4)*100)

In [ ]:
cm=metrics.confusion_matrix(Y_test,y_pred)
cm

In [ ]:
class_names=['Positive_review','Negative_review']
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
sns.heatmap(pd.DataFrame(cm), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')

In [ ]:
import pickle
pickle.dump(model,open('/content/sentimental_analysis','wb'))

In [ ]:
model_loaded=pickle.load(open('/content/sentimental_analysis','rb'))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer.fit(X_train)

X_test_transformed = vectorizer.transform(X_test)

model_loaded.predict(X_test_transformed)


In [ ]:
import joblib

In [ ]:
joblib.dump(model,'sentimental_analysis_model')

In [ ]:
S_A_M=joblib.load("/content/sentimental_analysis_model")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer.fit(X_train)

X_test_transformed = vectorizer.transform(X_test)

S_A_M.predict(X_test_transformed)


In [ ]:
pip install nltk

In [ ]:
joblib.dump(model, 'sentimental_analysis_model.joblib')


In [ ]:
import joblib
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier


In [ ]:
import joblib
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import os

model_filename = '/path/to/sentiment_analysis_model.joblib'
# Check if the model file exists
if os.path.exists(model_filename):
    # Load the model
    model = joblib.load(model_filename)

    # Define the predict_sentiment function
    def predict_sentiment(new_data, model):
        if not isinstance(model, RandomForestClassifier):
            model = joblib.load(model_filename)

        predictions = model.predict(new_data)
        return predictions

    # Get user input for review text
    review_text = input('Enter the Amazon review text: ')

    # Preprocess the input text
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(review_text)
    filtered_text = [word.lower() for word in word_tokens if word.isalnum() and word.lower() not in stop_words]
    processed_text = ' '.join(filtered_text)

    # Assuming CATEGORIES is a list of sentiment labels used during training
    CATEGORIES = ['negative', 'neutral', 'positive']

    # Vectorize the processed text using TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    text_vectorized = tfidf_vectorizer.transform([processed_text])

    # Make predictions and print the result
    y_out = predict_sentiment(text_vectorized, model)
    y_out = CATEGORIES[y_out[0]]
    print(f'PREDICTED SENTIMENT: {y_out}')

else:
    print(f"Error: Model file '{model_filename}' not found.")
